# Summary: This Notebook is used for MSK data preprocessing and prepare dataset for preditive models. It has three major steps:
## 1) Load knowledge graph (subgraph);
## 2) Load patient features and overall survival;
## 3) Patient graph embedding generation and save the dataset

In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from karateclub import SocioDim
from karateclub import EgoNetSplitter
import networkx as nx



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sksurv.util import Surv
from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from lifelines.utils import concordance_index as lfcindex
from sklearn.tree import DecisionTreeRegressor


In [ ]:
import random
randomSeed=10
np.random.seed(randomSeed);


# 1. Load hetionNet subgraph

## Here is to load the knowledge graph subgraph and prepare it in such format "source_label (e.g. gene) - target_label (e.g. gene, pathway)"

In [ ]:
whichSubgraph='WholeHetionNet'
subgraph_edges = pd.read_csv('../Data/inputs/HetionNetImmuneGeneGeneSubgraphs/wholeGraph.csv', sep=',')
subgraph_edges=subgraph_edges[['source_label','target_label']]
subgraph_edges


# 2. Load patient survival information

## This block is to load patient survival information, the patient survival information should in format of "SAMPLE_ID | OS_MONTHS | OS_STATUS", In OS_STATUS: 0-censored; 1-deceased

In [ ]:

clinical_features = pd.read_csv('../Data/inputDatasetMSKMET2021/clinical_features_Lung_Adenocarcinoma.csv', sep=',')
#clinical_subgroup = clinical_features[((clinical_features['SAMPLE_TYPE'].isin(['Metastasis']))&(clinical_features['Metastatic patient']==True))]
clinical_subgroup = clinical_features[((clinical_features['SAMPLE_TYPE'].isin(['Primary']))&(clinical_features['Metastatic patient']==True))]
#clinical_subgroup = clinical_features[((clinical_features['SAMPLE_TYPE'].isin(['Primary']))&(clinical_features['Metastatic patient']==False))]

#clinical_subgroup = clinical_features[clinical_features['SAMPLE_TYPE'].isin(['Metastasis'])]
survival_outcomes=clinical_subgroup[['SAMPLE_ID','OS_MONTHS','OS_STATUS']]
survival_outcomes.set_index('SAMPLE_ID',inplace=True)
survival_outcomes

In [ ]:
print (list(clinical_subgroup.columns))

## the following block is to load patient genomic mutation features

In [ ]:
genomic_features = pd.read_csv('../Data/inputDatasetMSKMET2021/genomic_features_Lung_Adenocarcinoma.csv', sep=',')
genomic_features=genomic_features[genomic_features['SAMPLE_ID'].isin(survival_outcomes.index)]
genomic_features.set_index('SAMPLE_ID',inplace=True)
genomic_features[genomic_features>0]=1
genomic_features

## Here is to split the dataset into training and hold out for testing. The random seed here is set for generating reperatable results

In [ ]:
# randomly select 10 percent samples from overall as holdout
### set up random state(a seed value) so that the results can be repeatable
holdout=genomic_features.sample(frac=0.1,random_state=randomSeed)
# the rest for model training/validation
dataframe=genomic_features.drop(holdout.index)

In [ ]:
len(dataframe)

In [ ]:
len(holdout)


# 3. Learn patient embedding using bikg_edges and gene_patient_edges

In [ ]:
subgraph_edges

## This function is used to convert patient genomic feature matrix into format of graph representation in format of "source_label (e.g. patient id) - target_label (e.g. gene mutation)"

In [ ]:
def getGenePatientEdges(genomic_features):
    (x_ind,y_ind)=np.where(genomic_features==1)
    gene_patient_edges=pd.DataFrame(list(zip(genomic_features.index[x_ind],genomic_features.columns[y_ind])),columns=['source_label','target_label'])
    return gene_patient_edges

In [ ]:
gene_patient_edges_holdout=getGenePatientEdges(holdout)
gene_patient_edges_dataframe=getGenePatientEdges(dataframe)

## The following function is to fuse "patient-gene" graph with knowledge graph subgraph containing of "gene-gene" or "gene-pathway"; Then the graph embedding algorithm (e.g. SocioDim) is applied to learn embedding for each gene node

In [ ]:

def learnBIKGGraphEmbeddings(subgraph_edges,gene_patient_edges):
    bikg_edges = pd.concat([subgraph_edges, gene_patient_edges])
    nodes = set(bikg_edges["source_label"].values.tolist()).union(
                set(bikg_edges["target_label"].values.tolist())
            )
    node_to_num = {index: node for node, index in enumerate(nodes)}
    bikg_edges["source_num_id"] = bikg_edges["source_label"].map(
                lambda x: node_to_num[x]
            )
    bikg_edges["target_num_id"] = bikg_edges["target_label"].map(
        lambda x: node_to_num[x]
    )
    bikg_edges = bikg_edges[["source_num_id", "target_num_id"]].values.tolist()
    dim=16
    upstream_model = SocioDim(dimensions=dim,seed=23)
    upstream_model.fit(nx.from_edgelist(bikg_edges))
    BIKG_graph_embedding = upstream_model.get_embedding()
    return BIKG_graph_embedding, node_to_num




In [ ]:
[BIKG_graph_embedding,node_to_num]=learnBIKGGraphEmbeddings(subgraph_edges,gene_patient_edges_dataframe)


## patient representation is generated as aggregation of gene embedding based on patient genomic mutation profile.

In [ ]:
def buildPatientEmbeddingUsingGeneEmbedding(gene_patient_edges,BIKG_graph_embedding,node_to_num):
    gene_patient_edges_grouped=gene_patient_edges.groupby('source_label')['target_label'].apply(list).reset_index(name='target_labels')
    dim=BIKG_graph_embedding.shape[1]
    patients_embeddings_feature=[]
    for index, row in gene_patient_edges_grouped.iterrows():
        target_labels=row['target_labels']
        patient_embeddings=[0]*dim
        target_labels_length=len(target_labels)
        for gene in target_labels:
            if gene in node_to_num:
                patient_embeddings=patient_embeddings+np.array(BIKG_graph_embedding[node_to_num[gene], :])
            else:
                patient_embeddings=patient_embeddings+np.array([0]*dim)
        patients_embeddings_feature.append(patient_embeddings/target_labels_length)
    emb_features = ['X{}'.format(i) for i in range(dim)]
    patients_embeddings_feature = pd.DataFrame(patients_embeddings_feature, index=gene_patient_edges_grouped['source_label'], columns = emb_features)
    return patients_embeddings_feature

In [ ]:
patient_embedding_holdout=buildPatientEmbeddingUsingGeneEmbedding(gene_patient_edges_holdout,BIKG_graph_embedding,node_to_num)
patient_embedding_dataframe=buildPatientEmbeddingUsingGeneEmbedding(gene_patient_edges_dataframe,BIKG_graph_embedding,node_to_num)



## save patient embedding results


In [ ]:
output_dir='../Data/outputs/EvaluationUsingHetionNet/'

patient_embedding_dataframe.to_csv(output_dir+whichSubgraph+'/'+str(randomSeed)+'/patient_embedding_dataframe.csv') 
patient_embedding_holdout.to_csv(output_dir+whichSubgraph+'/'+str(randomSeed)+'/patient_embedding_holdout.csv') 

In [ ]:
def getSurvivalInformation(survival_outcomes,patient_embedding):
    mergedDF=survival_outcomes.join(patient_embedding,how='right')
    y = Surv.from_dataframe("OS_STATUS","OS_MONTHS",  mergedDF)
    return y

In [ ]:
y_dataframe=getSurvivalInformation(survival_outcomes,patient_embedding_dataframe)
y_holdout=getSurvivalInformation(survival_outcomes,patient_embedding_holdout)

In [ ]:
np.save(output_dir+whichSubgraph+'/'+str(randomSeed)+'/y_dataframe.npy',y_dataframe) 
np.save(output_dir+whichSubgraph+'/'+str(randomSeed)+'/y_holdout.npy',y_holdout) 

In [ ]:
TMB=clinical_subgroup.set_index('SAMPLE_ID')[['TMB_NONSYNONYMOUS']]
filterTMB=TMB.join(patient_embedding_holdout,how='right')
tmbValue=list(filterTMB['TMB_NONSYNONYMOUS'])
np.save(output_dir+whichSubgraph+'/'+str(randomSeed)+'/tmbValue.npy',tmbValue) 

In [ ]:
genomic_features.to_csv(output_dir+whichSubgraph+'/'+str(randomSeed)+'/genomic_features.csv') 
